In [1]:
import numpy as np
import pandas as pd 
from pylab import mpl, plt
import random
import optuna
import os
os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
torch.set_num_threads(1)


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import warnings
warnings.simplefilter("ignore", UserWarning)

import vectorbtpro as vbt


from plotly.subplots import make_subplots
import plotly.graph_objects as go
from collections import deque

vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400


In [2]:
df_sol = pd.read_csv('2ySOLdata1h.csv')
df_sol['timestamp'] = pd.to_datetime(df_sol['timestamp'], unit='s')
df_sol.set_index('timestamp', inplace=True)
pd.set_option('future.no_silent_downcasting', True)

In [10]:
data = df_sol.iloc[:, 3:4]
data_trimmed = data[-100:].copy()

In [11]:
pd.set_option('future.no_silent_downcasting', True)
data_trimmed.loc[:, 'signal'] = 'SignalNone'
# Define window size
window_size = 10

rolling_max = data_trimmed.loc[:,'Close'].rolling(window=2*window_size+1, center=True, min_periods=1).max()
rolling_min = data_trimmed.loc[:,'Close'].rolling(window=2*window_size+1, center=True, min_periods=1).min()

is_peak = (data_trimmed.loc[:, 'Close'] == rolling_max)

is_low = (data_trimmed.loc[:, 'Close'] == rolling_min) 

# Update signal columns where conditions are met
data_trimmed.loc[is_peak, 'signal'] = 'SignalShort'  # Mark peaks with 0
data_trimmed.loc[is_low, 'signal'] = 'SignalLong'   # Mark lows with 0


In [5]:
df = data_trimmed.copy()

In [6]:
df_filtered = df[df['signal'] != 'SignalNone'].copy()

# Iterate through the DataFrame and adjust the signals
for i in range(1, len(df_filtered)):
    current_signal = df_filtered.iloc[i]['signal']
    previous_signal = df_filtered.iloc[i - 1]['signal']
    current_close = df_filtered.iloc[i]['Close']
    previous_close = df_filtered.iloc[i - 1]['Close']
    
    if current_signal == previous_signal:
        if current_signal == 'SignalLong' and previous_close > current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        elif current_signal != 'SignalLong' and previous_close < current_close:
            df_filtered.iloc[i - 1, df_filtered.columns.get_loc('signal')] = 'SignalNone'
        else:
            df_filtered.iloc[i, df_filtered.columns.get_loc('signal')] = 'SignalNone'


df.update(df_filtered)


# # Making binary... comment out if not desired
# previous_signal = None  # Initialize a variable to keep track of the previous non-"SignalNone" value

# for i in range(len(df)):
#     if df.iloc[i, df_filtered.columns.get_loc('signal')] == "SignalNone" and previous_signal is not None:
#         df.iloc[i, df_filtered.columns.get_loc('signal')] = previous_signal  # Replace "SignalNone" with the previous signal
#     elif df.iloc[i, df_filtered.columns.get_loc('signal')] != "SignalNone":
#         previous_signal = df.iloc[i, df_filtered.columns.get_loc('signal')]  # Update the previous signal to the current one if it's not "SignalNone"

# df = df.loc[df['signal'] != 'SignalNone']
# # end of binary

df.loc[:,'signal'] = df.loc[:,'signal'].replace({'SignalLong': 1, 'SignalShort': 0, 'SignalNone': 2})
df = df.ffill()

In [7]:
signal = df['signal']
entries = signal == 1
exits = signal == 0

In [8]:
pf = vbt.Portfolio.from_signals(
    close=df.Close, 
    long_entries=entries, 
    short_entries=exits,
    size=100,
    size_type='value',
    # accumulate=True,
    init_cash='auto'
)

In [9]:
pf.plot({"orders"}).show()